In [4]:
import json as json
import pathlib as pl
from copy import deepcopy
from datetime import datetime
from gc import collect
from os import path, system
from warnings import simplefilter

import matplotlib.pyplot as plt
from matplotlib import rcParams
from numpy import array, linspace, meshgrid, ones, zeros
from numpy.linalg import norm
from scipy.spatial.transform import Rotation
from tqdm import tqdm

from catenary import Catenary
from utils import check, gif_from_pngs, print_dict

simplefilter( 'ignore', RuntimeWarning )

gain = 1.5
rcParams[ 'figure.figsize' ] = [ gain * 6.4, gain * 4.8 / 2 ]
# rcParams['figure.facecolor'] = 'black'
# rcParams['axes.edgecolor'] = 'white'
# rcParams['axes.facecolor'] = 'black'
# rcParams['axes.labelcolor'] = 'white'
# rcParams['xtick.color'] = 'white'
# rcParams['ytick.color'] = 'white'
# rcParams['legend.labelcolor'] = 'white'
# rcParams['legend.facecolor'] = 'black'

In [5]:
files = list( pl.Path( "./export" ).glob( '*' ) )
files.sort( key = lambda x: path.getctime( x ) )
for v in files:
	print( str( v ).split( '//' )[ -1 ], end = '\t' )
	print( datetime.fromtimestamp( path.getctime( v ) ) )

export\three_robots_chain_1726157070	2024-09-17 09:04:31.536003
export\Figure_1.png	2024-10-10 09:58:20.942149
export\three_robots_chain_with_fixed_end_1726429003	2024-10-24 16:48:58.855255
export\three_robots_chain_1726514783	2024-10-24 19:05:24.019054
export\three_robots_chain_with_fixed_end_1726554218	2024-10-24 19:05:24.068337
export\chain_of_four_1730365663	2024-10-31 10:07:43.603700
export\chain_of_four_1730370647	2024-10-31 11:30:47.112706
export\chain_of_four_1730371997	2024-10-31 11:53:17.502967
export\chain_of_four_1730373858	2024-10-31 12:24:18.394288
export\chain_of_four_1730384599	2024-10-31 15:23:19.554600


In [6]:
folder = pl.Path( './export/chain_of_four_1730384599' )
uuid = str( folder ).split( '_' )[ -1 ]

In [7]:
with open( folder / 'config.json' ) as f:
	config = json.load( f )
print_dict( config )

predict: 	def _pred ...
get_actuation: 	def _get_ ...
get_result: 	def _get_ ...
model:
	dynamics:
		br_0:
			state_size: 12
			pose_size: 6
			actuation_size: 4
			linear_actuation_size: 3
			build_transformation_matrix:
				instance_of: staticmeth ...
			build_inertial_matrix:
				instance_of: staticmeth ...
			mass: 11.5
			center_of_mass: [0.0, 0.0, 0.0]
			weight: [0.0, 0.0, 112.77647499999999]
			volume: 0.0134
			center_of_volume: [0.0, 0.0, -0.01]
			buoyancy: [-0.0, -0.0, -131.01488267000002]
			water_surface_depth: 0.0
			water_current: [0.5, 0.5, 0.0, 0.0, 0.0, 0.0]
			inertial_coefficients: [0.26, 0.23, 0.37, 0.0, 0.0, 0.0]
			hydrodynamic_coefficients: [13.7, 0.0, 33.0, 0.0, 0.8, 0.0]
			added_mass_coefficients: [6.36, 7.12, 18.68, 0.189, 0.135, 0.222]
			inertial_matrix: (6, 6)
			inverse_inertial_matrix: (6, 6)
			hydrodynamic_matrix: (6, 6)
			instance_of: BluerovXYZ ...
		c_01:
			length: 3.0
			linear_mass: 0.01
			optimization_function:
				instance_of: staticmeth ..

In [8]:
slice_repository = { }
axes = [ 'x', 'y', 'z', 'phi', 'theta', 'psi', 'u', 'v', 'w', 'p', 'q', 'r' ]

for k, v in config[ 'model' ][ 'dynamics' ].items():
	if isinstance( v, str ) and '\n' not in v and ':' in v:
		slice_repository[ k ] = slice( *[ int( v ) if v != 'None' else None for v in v.split( ':' ) ] )
	elif isinstance( v, int ) and 'br_' in k:
		slice_repository[ k ] = v

pose = { k: v for k, v in slice_repository.items() if 'pose' in k }
positions = { k: v for k, v in slice_repository.items() if 'position' in k }
orientations = { k: v for k, v in slice_repository.items() if 'orientation' in k }
positions_pairs = { '_'.join( a.split( '_' )[ :2 ] ): '_'.join( b.split( '_' )[ :2 ] ) for a, b in
										zip( list( positions )[ :-1 ], list( positions )[ 1: ] ) }
speeds = { k: v for k, v in slice_repository.items() if 'linear_speed' in k }
linear_actuation = { k: v for k, v in slice_repository.items() if 'linear_actuation' in k }
angular_actuation = { k: v for k, v in slice_repository.items() if 'angular_actuation' in k }

In [9]:
for i, c in enumerate( config[ 'constraints' ] ):
	for j, (lb, ub, label, vlabel) in enumerate( zip( c[ 'lb' ], c[ 'ub' ], c[ 'labels' ], c[ 'value_labels' ] ) ):
		print( f'{(i + 1) * j}.\t{label}: {lb}\t<=\t{vlabel}\t<=\t{ub}' )

0.	seafloor: 0.2	<=	c_01_distance_to_seafloor	<=	inf
1.	seafloor: 0.2	<=	c_12_distance_to_seafloor	<=	inf
2.	seafloor: 0.2	<=	c_23_distance_to_seafloor	<=	inf
3.	seafloor: 0.2	<=	br_0_distance_to_seafloor	<=	inf
4.	seafloor: 0.2	<=	br_1_distance_to_seafloor	<=	inf
5.	seafloor: 0.2	<=	br_2_distance_to_seafloor	<=	inf
6.	seafloor: 0.2	<=	br_3_distance_to_seafloor	<=	inf
7.	cable_length: 0.2	<=	br_0_br_1_horizontal_distance	<=	2.8
8.	cable_length: 0.2	<=	br_1_br_2_horizontal_distance	<=	2.8
9.	cable_length: 0.2	<=	br_2_br_3_horizontal_distance	<=	2.8
10.	cable_length: -inf	<=	br_0_br_1_distance	<=	2.8
11.	cable_length: -inf	<=	br_1_br_2_distance	<=	2.8
12.	cable_length: -inf	<=	br_2_br_3_distance	<=	2.8


In [10]:
data_files = list( folder.glob( 'data/*' ) )
data_files.sort( key = lambda x: path.getmtime( x ) )
with open( data_files[ -1 ] ) as f:
	final_state = json.load( f )

times = [ config[ 'model' ][ 'time_step' ] * i for i in range( len( config[ 'target_trajectory' ] ) ) ]
target_trajectory = array( config[ 'target_trajectory' ] )[ :, 0 ]

n_frames = len( data_files )
previous_times = [ config[ 'model' ][ 'time_step' ] * i for i in range( n_frames ) ]
previous_target_trajectory = target_trajectory[ :n_frames ]
previous_actual_trajectory = array( final_state[ 'model' ][ 'previous_states' ] )[ 1:,
														 :config[ 'model' ][ 'dynamics' ][ 'state_size' ] ]

previous_actuations = array( final_state[ 'model' ][ 'previous_actuations' ] )

pose_weight = array( config[ 'pose_weight_matrix' ][ 0 ] )

In [11]:
check( f'{folder}/plots' )

0

In [12]:
for k, v in positions.items():
	if norm( pose_weight[ v, v ] ) == 0:
		continue
	absolute_distances = abs( previous_target_trajectory[ :, v ] - previous_actual_trajectory[ :, v ] )
	plt.plot( previous_times, absolute_distances )
	plt.legend(
			[
					r'over $\mathbf{x}_w$-axis',
					r'over $\mathbf{y}_w$-axis',
					r'over $\mathbf{z}_w$-axis'
					]
			)
	plt.xlabel( 'time [s]' )
	plt.ylabel( 'absolute error [m]' )
	plt.subplots_adjust( bottom = 0.125, top = 1, left = 0.075, right = 1 )
	plt.savefig( f'{folder}/plots/{k}_tracking_error_{uuid}.png', dpi = 300 )
	plt.close( 'all' )

In [13]:
for k, v in orientations.items():
	if norm( pose_weight[ v, v ] ) == 0:
		continue
	plt.plot( previous_times, abs( previous_target_trajectory[ :, v ] - previous_actual_trajectory[ :, v ] ) )
	plt.legend(
			[
					r'around $\mathbf{x}_w$-axis',
					r'around $\mathbf{y}_w$-axis',
					r'around $\mathbf{z}_w$-axis'
					]
			)
	plt.xlabel( 'time [s]' )
	plt.ylabel( 'absolute error [rad]' )
	plt.subplots_adjust( bottom = 0.125, top = 1, left = 0.075, right = 1 )
	plt.savefig( f'{folder}/plots/{k}_tracking_error_{uuid}.png', dpi = 300 )
	plt.close( 'all' )

In [14]:
for k, v in linear_actuation.items():
	plt.plot( previous_times, previous_actuations[ :-1, v ] )
	plt.xlabel( 'time [s]' )
	plt.ylabel( 'actuation [N]' )
	plt.legend(
			[
					r'over $\mathbf{x}_r$-axis',
					r'over $\mathbf{y}_r$-axis',
					r'over $\mathbf{z}_r$-axis'
					]
			)
	plt.subplots_adjust( bottom = 0.125, top = 1, left = 0.075, right = 1 )
	plt.savefig( f'{folder}/plots/{k}_{uuid}.png', dpi = 300 )
	plt.close( 'all' )

In [15]:
for k, v in angular_actuation.items():
	plt.plot( previous_times, previous_actuations[ :-1, v ] )
	plt.xlabel( 'time [s]' )
	plt.ylabel( 'actuation [Nm]' )
	plt.legend(
			[
					r'around $\mathbf{x}_r$-axis',
					r'around $\mathbf{y}_r$-axis',
					r'around $\mathbf{z}_r$-axis'
					]
			)
	plt.subplots_adjust( bottom = 0.125, top = 1, left = 0.075, right = 1 )
	plt.savefig( f'{folder}/plots/{k}_{uuid}.png', dpi = 300 )
	plt.close( 'all' )

In [16]:
for pair_1, pair_2 in positions_pairs.items():
	position_1 = deepcopy( previous_actual_trajectory[ :, slice_repository[ pair_1 + '_position' ] ] )
	position_2 = deepcopy( previous_actual_trajectory[ :, slice_repository[ pair_2 + '_position' ] ] )

	lowest_points = zeros( (position_1.shape[ 0 ],) )

	name = f'c_{pair_1.split( "_" )[ -1 ]}{pair_2.split( "_" )[ -1 ]}'
	cat = Catenary(
			config[ 'model' ][ 'dynamics' ][ name ][ 'length' ],
			config[ 'model' ][ 'dynamics' ][ name ][ 'linear_mass' ]
			)

	for i, (p1, p2) in enumerate( zip( position_1, position_2 ) ):
		lowest_points[ i ] = cat.get_lowest_point( p1, p2 )[ 2 ]

	plt.plot( previous_times, lowest_points )

plt.legend( [ f'from {a} to {b}' for a, b in positions_pairs.items() ] )
plt.xlabel( 'time [s]' )
plt.ylabel( 'distance [m]' )
plt.subplots_adjust( bottom = 0.125, top = 1, left = 0.075, right = 1 )
plt.gca().invert_yaxis()
plt.savefig( f'{folder}/plots/lowest_points_{uuid}.png', dpi = 300 )
plt.close( 'all' )

In [17]:
for pair_1, pair_2 in positions_pairs.items():
	absolute_distances = norm(
			previous_actual_trajectory[ :, positions[ pair_1 + '_position' ] ] -
			previous_actual_trajectory[ :, positions[ pair_2 + '_position' ] ],
			axis = 1
			)
	plt.plot( previous_times, absolute_distances )
plt.legend( [ f'from {a} to {b}' for a, b in positions_pairs.items() ] )
plt.xlabel( 'time [s]' )
plt.ylabel( 'distance [m]' )
plt.subplots_adjust( bottom = 0.125, top = 1, left = 0.075, right = 1 )
plt.savefig( f'{folder}/plots/distance_{uuid}.png', dpi = 300 )
plt.close( 'all' )

In [18]:
for p1, p2 in positions_pairs.items():
	absolute_distances = norm(
			previous_actual_trajectory[ :, slice_repository[ p1 + '_xy' ] ] -
			previous_actual_trajectory[ :, slice_repository[ p2 + '_xy' ] ],
			axis = 1
			)
	plt.plot( previous_times, absolute_distances )
plt.legend( [ f'from {a} to {b}' for a, b in positions_pairs.items() ] )
plt.xlabel( 'time [s]' )
plt.ylabel( 'distance [m]' )
plt.subplots_adjust( bottom = 0.125, top = 1, left = 0.075, right = 1 )
plt.savefig( f'{folder}/plots/horizontal_distance_{uuid}.png', dpi = 300 )
plt.close( 'all' )

In [19]:
figs = [ 'x', 'y', 'z' ]

x = plt.figure().subplots()
y = plt.figure().subplots()
z = plt.figure().subplots()
z.invert_yaxis()

legends = [ [ ], [ ], [ ] ]

for k, v in positions.items():
	position = previous_actual_trajectory[ :, v ]

	X = position[ :, 0 ]
	Y = position[ :, 1 ]
	Z = position[ :, 2 ]

	x.plot( previous_times, X )
	y.plot( previous_times, Y )
	z.plot( previous_times, Z )

	for i in range( 3 ):
		legends[ i ] += [ f'{k}' ]

	if norm( pose_weight[ v, v ][ :, 0 ] ) > 0:
		target = previous_target_trajectory[ :, v ][ :, 0 ]
		x.plot( previous_times, target, ':', linewidth = 3 )
		legends[ 0 ] += [ f'{k} target' ]

	if norm( pose_weight[ v, v ][ :, 1 ] ) > 0:
		target = previous_target_trajectory[ :, v ][ :, 1 ]
		y.plot( previous_times, target, ':', linewidth = 3 )
		legends[ 1 ] += [ f'{k} target' ]

	if norm( pose_weight[ v, v ][ :, 2 ] ) > 0:
		target = previous_target_trajectory[ :, v ][ :, 2 ]
		z.plot( previous_times, target, ':', linewidth = 3 )
		legends[ 2 ] += [ f'{k} target' ]

for f in plt.get_fignums():
	fig = plt.figure( f )
	plt.legend( legends[ f - 1 ] )
	plt.xlabel( 'time [s]' )
	plt.ylabel( f'position on $\\mathbf{{{figs[ f - 1 ]}}}_w$-axis [m]' )
	plt.subplots_adjust( bottom = 0.125, top = 1, left = 0.075, right = 1 )
	plt.savefig( f'{folder}/plots/positions_on_{figs[ f - 1 ]}_{uuid}.png', dpi = 300 )
plt.close( 'all' )

In [20]:
figs = [ 'x', 'y', 'z' ]

x = plt.figure().subplots()
y = plt.figure().subplots()
z = plt.figure().subplots()
z.invert_yaxis()

legends = [ [ ], [ ], [ ] ]

for k, v in speeds.items():
	speed = previous_actual_trajectory[ :, v ]

	X = speed[ :, 0 ]
	Y = speed[ :, 1 ]
	Z = speed[ :, 2 ]

	x.plot( previous_times, X )
	y.plot( previous_times, Y )
	z.plot( previous_times, Z )

	for i in range( 3 ):
		legends[ i ] += [ f'{k}' ]

for f in plt.get_fignums():
	fig = plt.figure( f )
	plt.legend( legends[ f - 1 ] )
	plt.xlabel( 'time [s]' )
	plt.ylabel( f'speed on $\\mathbf{{{figs[ f - 1 ]}}}_r$-axis [m/s]' )
	plt.subplots_adjust( bottom = 0.125, top = 1, left = 0.075, right = 1 )
	plt.savefig( f'{folder}/plots/speeds_on_{figs[ f - 1 ]}_{uuid}.png', dpi = 300 )
plt.close( 'all' )

In [21]:
plt.plot( previous_times, final_state[ 'compute_times' ] )
plt.hlines( config[ 'time_step' ] * config[ 'horizon' ], 0, previous_times[ -1 ], 'r', '--' )
plt.legend( [ 'compute time', 'horizon depth' ] )
plt.xlabel( 'time [s]' )
plt.ylabel( 'time [s]' )
plt.subplots_adjust( bottom = 0.125, top = 1, left = 0.075, right = 1 )
plt.savefig( f'{folder}/plots/compute_times_{uuid}.png', dpi = 300 )
plt.close( 'all' )

In [22]:
frames_already_done = check( f'{folder}/plots/3d_frames' )

In [23]:
robots_colors = { '0': 'r', '1': 'g', '2': 'b', '3': 'y', '4': 'm' }
quiver_scale = .5
box = [ [ -6.5, 4.5 ], [ -5.5, 5.5 ], [ -5, 6 ] ]

water_current = config[ 'model' ][ 'dynamics' ][ 'br_0' ][ 'water_current' ]
water_surface = config[ 'model' ][ 'dynamics' ][ 'br_0' ][ 'water_surface_depth' ]
seafloor_config = config[ 'model' ][ 'dynamics' ][ 'sf' ]
if seafloor_config[ 'instance_of' ] == 'SeafloorFromFunction':
	seafloor_function = None
	exec( seafloor_config[ 'seafloor_function' ] )
else:
	raise NotImplementedError

Xc = linspace( box[ 0 ][ 0 ], box[ 0 ][ 1 ], 5 )
Yc = linspace( box[ 1 ][ 0 ], box[ 1 ][ 1 ], 5 )
Zc = linspace(
		box[ 2 ][ 0 ] if box[ 2 ][ 0 ] > water_surface else water_surface,
		box[ 2 ][ 1 ],
		5
		)

Xc, Yc, Zc = meshgrid( Xc, Yc, Zc )

Uc = water_current[ 0 ] * ones( Xc.shape )
Vc = water_current[ 1 ] * ones( Xc.shape )
Wc = water_current[ 2 ] * ones( Xc.shape )

Xs = linspace( box[ 0 ][ 0 ], box[ 0 ][ 1 ], 1000 )
Ys = linspace( box[ 1 ][ 0 ], box[ 1 ][ 1 ], 1000 )
Xg, Yg = meshgrid( Xs, Ys )
Zws = water_surface * ones( Xg.shape )
Zsf = zeros( Xg.shape )
Zsfx = zeros( Xs.shape )
Zsfy = zeros( Ys.shape )

for i, y in enumerate( Ys ):
	Zsfy[ i ] = seafloor_function( 0, y )
	for j, x in enumerate( Xs ):
		Zsf[ i, j ] = seafloor_function( x, y )
		if i == 0:
			Zsfx[ j ] = seafloor_function( x, 0 )

for frame in tqdm( range( frames_already_done, n_frames ) ):
	# frame = 100

	with open( data_files[ frame ] ) as f:
		simulation_state = json.load( f )
	predicted_trajectories = array( simulation_state[ 'predicted_trajectories' ] )
	if predicted_trajectories.shape[ 0 ] > 100:
		predicted_trajectories = predicted_trajectories[ ::predicted_trajectories.shape[ 0 ] // 100 ]

	fig = plt.figure( figsize = (10, 10) )
	view = plt.subplot( projection = '3d', computed_zorder = False )
	view.set_xlabel( r"$\mathbf{x}_w$-axis" )
	view.set_ylabel( r"$\mathbf{y}_w$-axis" )
	view.set_zlabel( r"$\mathbf{z}_w$-axis" )
	view.set_xlim( *box[ 0 ] )
	view.set_ylim( *box[ 1 ] )
	view.set_zlim( *box[ 2 ] )
	view.invert_yaxis()
	view.invert_zaxis()

	inset_view_xz = view.inset_axes( [ .0, .0, .2, .2 ] )
	inset_view_xz.set_xlabel( r"$\mathbf{x}_w$-axis" )
	inset_view_xz.set_ylabel( r"$\mathbf{z}_w$-axis" )
	inset_view_xz.set_xlim( *box[ 0 ] )
	inset_view_xz.set_ylim( *box[ 2 ] )
	inset_view_xz.invert_yaxis()

	inset_view_yz = view.inset_axes( [ .8, .0, .2, .2 ] )
	inset_view_yz.set_xlabel( r"$\mathbf{y}_w$-axis" )
	inset_view_yz.set_ylabel( r"$\mathbf{z}_w$-axis" )
	inset_view_yz.set_xlim( *box[ 1 ] )
	inset_view_yz.set_ylim( *box[ 2 ] )
	inset_view_yz.invert_yaxis()
	inset_view_yz.invert_xaxis()

	inset_view_xy = view.inset_axes( [ .0, .8, .2, .2 ] )
	inset_view_xy.set_xlabel( r"$\mathbf{x}_w$-axis" )
	inset_view_xy.set_ylabel( r"$\mathbf{y}_w$-axis" )
	inset_view_xy.set_xlim( *box[ 0 ] )
	inset_view_xy.set_ylim( *box[ 1 ] )
	inset_view_xy.invert_yaxis()

	view.quiver( Xc, Yc, Zc, Uc, Vc, Wc, color = 'navy' )
	view.plot_surface( Xg, Yg, Zsf, color = 'brown', zorder = 0. )
	view.plot_surface( Xg, Yg, Zws, color = 'navy', alpha = 0.1 )

	inset_view_xz.plot( Xs, Zsfx, color = 'brown', zorder = 0. )
	inset_view_yz.plot( Ys, Zsfy, color = 'brown', zorder = 0. )
	inset_view_xz.axhline( water_surface, color = 'navy' )
	inset_view_yz.axhline( water_surface, color = 'navy' )

	for k, v in pose.items():

		color = robots_colors[ k.split( '_' )[ 1 ] ]

		if norm( pose_weight[ v, v ][ :3, :3 ] ) > 0:
			Xt = target_trajectory[ :, v ][ :, 0 ]
			Yt = target_trajectory[ :, v ][ :, 1 ]
			Zt = target_trajectory[ :, v ][ :, 2 ]

			view.plot( Xt, Yt, Zt, ':', color = 'k' )
			view.scatter( Xt[ frame + 1 ], Yt[ frame + 1 ], Zt[ frame + 1 ], marker = 'x', color = 'k' )
			inset_view_xz.plot( Xt, Zt, ':', color = 'k' )
			inset_view_xz.scatter( Xt[ frame + 1 ], Zt[ frame + 1 ], marker = 'x', color = 'k' )
			inset_view_yz.plot( Yt, Zt, ':', color = 'k' )
			inset_view_yz.scatter( Yt[ frame + 1 ], Zt[ frame + 1 ], marker = 'x', color = 'k' )
			inset_view_xy.plot( Xt, Yt, ':', color = 'k' )
			inset_view_xy.scatter( Xt[ frame + 1 ], Yt[ frame + 1 ], marker = 'x', color = 'k' )

		X = previous_actual_trajectory[ frame, v ][ 0 ]
		Y = previous_actual_trajectory[ frame, v ][ 1 ]
		Z = previous_actual_trajectory[ frame, v ][ 2 ]

		P = previous_actual_trajectory[ frame, v ][ :3 ]
		A = previous_actual_trajectory[ frame, v ][ 3:6 ]
		R = Rotation.from_euler( 'xyz', A ).as_matrix()

		view.quiver( *P, *(R @ array( [ quiver_scale, 0., 0. ] )), color = 'r' )
		view.quiver( *P, *(R @ array( [ 0., quiver_scale, 0. ] )), color = 'g' )
		view.quiver( *P, *(R @ array( [ 0., 0., quiver_scale ] )), color = 'b' )

		view.scatter( X, Y, Z, color = color )
		inset_view_xz.scatter( X, Z, color = color )
		inset_view_yz.scatter( Y, Z, color = color )
		inset_view_xy.scatter( X, Y, color = color )

		for trajectory in predicted_trajectories:
			X = trajectory[ :, 0, v ][ :, 0 ]
			Y = trajectory[ :, 0, v ][ :, 1 ]
			Z = trajectory[ :, 0, v ][ :, 2 ]

			view.plot( X, Y, Z, linewidth = .5, color = color )
			inset_view_xz.plot( X, Z, linewidth = .5, color = color )
			inset_view_yz.plot( Y, Z, linewidth = .5, color = color )
			inset_view_xy.plot( X, Y, linewidth = .5, color = color )

	for p1, p2 in positions_pairs.items():

		color = robots_colors[ p1.split( '_' )[ 1 ] ]

		position_1 = deepcopy( previous_actual_trajectory[ frame, positions[ p1 + '_position' ] ] )
		position_2 = deepcopy( previous_actual_trajectory[ frame, positions[ p2 + '_position' ] ] )

		name = f'c_{p1.split( "_" )[ -1 ]}{p2.split( "_" )[ -1 ]}'
		cat = Catenary(
				config[ 'model' ][ 'dynamics' ][ name ][ 'length' ],
				config[ 'model' ][ 'dynamics' ][ name ][ 'linear_mass' ]
				)

		cat_12 = cat.discretize( position_1, position_2 )

		view.plot( cat_12[ :, 0 ], cat_12[ :, 1 ], cat_12[ :, 2 ], color = color )
		inset_view_xz.plot( cat_12[ :, 0 ], cat_12[ :, 2 ], color = color )
		inset_view_yz.plot( cat_12[ :, 1 ], cat_12[ :, 2 ], color = color )
		inset_view_xy.plot( cat_12[ :, 0 ], cat_12[ :, 1 ], color = color )

	plt.savefig( f'{folder}/plots/3d_frames/{frame}.png', dpi = 100 )
	plt.close( 'all' )
	del fig, view, inset_view_xz, inset_view_yz, inset_view_xy

100%|██████████| 200/200 [10:11<00:00,  3.06s/it]


In [24]:
collect()

25674874

In [25]:
gif_from_pngs( f'{folder}/plots/3d_frames', duration = config[ 'model' ][ 'time_step' ] * 1000 )

In [26]:
collect()

0

In [27]:
system( f'ffmpeg -y -i {folder}/plots/3d_frames/animation.gif {folder}/plots/animation.mp4' )

0